In [1]:
from Functions_6 import normalize_data, time_series_CV_split, wrapper_feature_selector, train_and_predict, fine_tune_enet, warn
from pyspark import SparkContext
from spark_sklearn import GridSearchCV
from sklearn.pipeline import Pipeline
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.preprocessing import MinMaxScaler 
from time import time
from math import sqrt
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from matplotlib import pyplot
import plotly.offline as py
import plotly.graph_objs as go
import warnings
warnings.warn = warn
py.init_notebook_mode(connected=True)
%matplotlib inline

# Train Set
train_data = pd.read_csv('/Users/Andy/Desktop/Bitcoin/Data/Train_data_after_EDA.csv')
print('n_features:', len(train_data.iloc[0]))
print('n_samples:', len(train_data))
train_data.head()


/anaconda3/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


n_features: 11
n_samples: 365


,date,close,volume,BCHAIN-DIFF,BCHAIN-AVBLS,BCHAIN-MIREV,BCHAIN-CPTRA,BCHAIN-NTRAN,BCHAIN-HRATE,BCHAIN-CPT,BCHAIN-NTRBL
0,1/1/18,13657.2,10291200000,1.870000e+12,1.037057,35435185.86,125.970638,290422,15177350.25,122.012747,1781.730061
1,2/1/18,14982.1,16846600192,1.920000e+12,1.043383,32334328.99,138.227164,241757,14975580.96,133.747230,1549.724359
2,3/1/18,15201.0,16871900160,1.930000e+12,1.041368,40553327.64,122.635624,340980,16415540.67,118.931690,1994.035088
3,4/1/18,15599.2,21783199744,1.930000e+12,1.065513,39612658.08,103.108719,395963,15071578.27,100.041312,2522.057325
4,5/1/18,17429.5,23840899072,1.930000e+12,1.065833,42527795.29,102.933856,425008,16127548.73,100.063517,2529.809524


In [2]:
# Test Set
test_data = pd.read_csv('/Users/Andy/Desktop/Bitcoin/Data/Test_data_after_EDA.csv')
print('n_features:', len(test_data.iloc[0]))
print('n_samples:', len(test_data))
test_data.head()

n_features: 11
n_samples: 181


,date,close,volume,BCHAIN-DIFF,BCHAIN-AVBLS,BCHAIN-MIREV,BCHAIN-CPTRA,BCHAIN-NTRAN,BCHAIN-HRATE,BCHAIN-CPT,BCHAIN-NTRBL
0,1/1/19,3843.52,4324200990,5.250000e+12,0.888394,7406437.589,28.520962,259684,43291796.76,28.520962,1675.380645
1,2/1/19,3943.41,5244856836,5.620000e+12,0.801779,7030739.129,29.953090,234725,41615985.27,29.953090,1575.335570
2,3/1/19,3836.74,4530215219,5.620000e+12,0.947861,7368988.356,27.122182,271696,42174589.10,27.122182,1799.311258
3,4/1/19,3857.72,4847965467,5.620000e+12,0.966222,7564081.637,25.991979,291016,43291796.76,25.991979,1877.522581
4,5/1/19,3845.19,5137609824,5.620000e+12,0.959199,7194486.330,25.533007,281772,41615985.27,25.533007,1891.087248


In [3]:
combined_data = pd.concat([train_data, test_data], ignore_index=True) # use data from 01/2018 onwards only - best stationarity
combined_data['Price'] = combined_data['close'].shift(-1) # Dependent variable Y

In [4]:
# Combined train and test sets
combined_data = pd.concat([train_data, test_data], ignore_index=True) # use data from 01/2018 onwards only - best stationarity
combined_data['Price'] = combined_data['close'].shift(-1) # Dependent variable Y
print('n_features:', len(combined_data.iloc[0]))
print('n_samples:', len(combined_data))
combined_data = combined_data.iloc[:-1,1:]
combined_data.tail()

n_features: 12
n_samples: 546


,close,volume,BCHAIN-DIFF,BCHAIN-AVBLS,BCHAIN-MIREV,BCHAIN-CPTRA,BCHAIN-NTRAN,BCHAIN-HRATE,BCHAIN-CPT,BCHAIN-NTRBL,Price
540,11790.92,24879684533,7.410000e+12,1.215810,25117669.41,64.679250,388342,65193212.50,64.679250,2194.022599,13016.23
541,13016.23,45105733173,7.410000e+12,1.245764,22019741.67,60.931532,361385,53775192.23,60.931532,2475.239726,11182.81
542,11182.81,39977475223,7.410000e+12,1.240849,28500796.50,71.319031,399624,60773333.69,71.319031,2421.963636,12407.33
543,12407.33,35087757766,7.870000e+12,1.233318,23373593.45,64.737830,361050,56010016.17,64.737830,2542.605634,11959.37
544,11959.37,29923961128,7.930000e+12,1.255488,25894830.35,63.608971,407094,64293187.57,63.608971,2497.509202,10817.16


In [6]:
# Feature Meta Subset
subset = [8, 7, 6, 5, 4, 3, 2, 1, 0] 

# Split train data into X (features) and Y (dependent variable)
data = combined_data.values 
Y_train = data[:-181,-1].reshape(-1,1) 
X_train = data[:-181,:-1]

# Training Validation samples size (1/4/18 - 30/6/18)
n_validation = 91

In [ ]:
# Fine-tune alpha
best_alpha, best_l1_ratio = fine_tune_enet(X_train[:,subset],Y_train)
print('Best alpha:',str(best_alpha))
print('Best l1 ratio:',str(best_l1_ratio))

# Feature Selection
selected_features = wrapper_feature_selector(X_train,Y_train,ElasticNet(l1_ratio=best_l1_ratio,alpha=best_alpha),subset)[0]   
print('Selected Features:',str(selected_features))


In [ ]:
%%time
rmse,Y_train_test,Y_train_pred= train_and_predict(X_train[:,selected_features],Y_train,ElasticNet(l1_ratio=best_l1_ratio,alpha=best_alpha),predict=False)
print('Train RMSE: {:0.2f}'.format(rmse))

In [ ]:
%%time
# Test samples size (1/1/19 - 30/6/19)
n_validation = 179

# Split test data into X (features) and Y (dependent variable)
Y_test = data[:,-1].reshape(-1,1) # including train data for fitting the model
X_test = data[:,:-1]





In [ ]:
rmse,Y_test,Y_pred= train_and_predict(X_test,Y_test,ElasticNet(l1_ratio=best_l1_ratio,alpha=best_alpha),n_validation,subset,predict=True)
print('Test RMSE: {:0.2f}'.format(rmse))



In [ ]:
# Output prediction to csv file
output = {'Y_pred': Y_pred.reshape(-1,)}
output = DataFrame(output)
output.to_csv('ElasticNet.csv', index=False)


In [ ]:
trace1 = go.Scatter(
    x = np.arange(0, len(Y_pred), 1),
    y = Y_pred.reshape(-1,),
    mode = 'lines',
    name = 'Predicted labels',
    line = dict(color=('rgb(244, 146, 65)'), width=2)
)
trace2 = go.Scatter(
    x = np.arange(0, len(Y_test), 1),
    y = Y_test.reshape(-1,),
    mode = 'lines',
    name = 'True labels',
    line = dict(color=('rgb(66, 244, 155)'), width=2)
)

layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our Ridge model predicted',
             xaxis = dict(title = 'Day number'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=[trace1, trace2], layout=layout)
py.iplot(fig, filename='results_demonstrating0')

